# spark RDD

In [1]:
import findspark
findspark.init()
import pyspark

In [9]:
# conf = pyspark.SparkConf().setAppName("My First Spark RDD APP").setMaster("local")  # “yarn”
# sc = pyspark.SparkContext(conf=conf)
sc = pyspark.SparkContext("yarn", "My First Spark App")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=My First Spark App, master=yarn) created by __init__ at /tmp/ipykernel_28869/3844447949.py:3 

In [3]:
from pyspark.mllib.linalg import Matrix, Matrices
# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm2 = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])
print(dm2)
# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])
print(sm)

DenseMatrix([[1., 4.],
             [2., 5.],
             [3., 6.]])
3 X 2 CSCMatrix
(0,0) 9.0
(2,1) 6.0
(1,1) 8.0


In [3]:
# sc.stop()
# sc = pyspark.SparkContext.getOrCreate()

In [5]:
!ls airdelay

airdelay_small.csv  data_info.csv  dummy_info.pkl  dummy_info_small.pkl


In [13]:
textFile = sc.textFile("/lifeng/student/liutuozhen/airdelay_small.csv") # load from hdfs
textFile.first()

'Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay'

In [14]:
textFile.count()

5548755

In [17]:
lineLengths = textFile.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)

In [18]:
lineLengths

PythonRDD[23] at RDD at PythonRDD.scala:53

In [19]:
totalLength

534280797

# spark df

SparkSession是Spark2.x后引入的概念。在2.x之前，对于不同的功能，需要使用不同的Context：


* 创建和操作RDD时，使用SparkContext

* 使用Streaming时，使用StreamingContext

* 使用SQL时，使用sqlContext

* 使用Hive时，使用HiveContext


在2.x中，为了统一上述的Context，引入SparkSession，实质上是SQLContext、HiveContext、SparkContext的组合。

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark").master('yarn').getOrCreate()

log4j:WARN No such property [datePattern] in org.apache.log4j.RollingFileAppender.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/01 19:31:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/01 19:31:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/01 19:31:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/12/01 19:31:39 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
sc = spark.sparkContext # for RDD

In [4]:
spark

In [5]:
from pyspark.mllib.linalg.distributed import RowMatrix, BlockMatrix

In [6]:
import numpy as np

In [15]:
x = RowMatrix(sc.parallelize([[1,1,3],[4,2,6],[6,4,9]]))
y = RowMatrix(sc.parallelize([1,3,6]))

In [16]:
# xTx = x.computeGramianMatrix()

In [17]:
# xTx

DenseMatrix(3, 3, [53.0, 33.0, 81.0, 33.0, 21.0, 51.0, 81.0, 51.0, 126.0], 0)

In [20]:
xy = RowMatrix(sc.parallelize([[1,1,3,1],[4,2,6,3],[6,4,9,6]]))

In [54]:
xyTxy = xy.computeGramianMatrix().toArray().reshape(4,4)

In [42]:
xTy = xyTxy[:-1, -1]

In [43]:
xTy

array([49., 31., 75.])

In [44]:
xTx = xyTxy[:-1, :-1]

In [45]:
xTx

array([[ 53.,  33.,  81.],
       [ 33.,  21.,  51.],
       [ 81.,  51., 126.]])

In [46]:
beta_hat = np.linalg.solve(xTx, xTy)

In [47]:
beta_hat

array([ 0.5       ,  1.5       , -0.33333333])

In [48]:
xyTxy = xy.computeGramianMatrix()

In [53]:
aaa = xyTxy.toArray()

In [55]:
aaa

array([[ 53.,  33.,  81.,  49.],
       [ 33.,  21.,  51.,  31.],
       [ 81.,  51., 126.,  75.],
       [ 49.,  31.,  75.,  46.]])

In [25]:
xyTxy.multiply(DenseMatrix(4, 1, [0, 0, 0, 1]))

AttributeError: 'DenseMatrix' object has no attribute 'multiply'

In [26]:
xyTxy.values

array([ 53.,  33.,  81.,  49.,  33.,  21.,  51.,  31.,  81.,  51., 126.,
        75.,  49.,  31.,  75.,  46.])

In [ ]:
rm.multiply(DenseMatrix(2, 2, [0, 2, 1, 3])).rows.collect()

In [8]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
distData

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [9]:
dist = sc.textFile("/lifeng/student/liutuozhen/streaming/aggregate/table.csv")
dist

/lifeng/student/liutuozhen/streaming/aggregate/table.csv MapPartitionsRDD[2] at textFile at NativeMethodAccessorImpl.java:0

## Spark-shell

In [10]:
textFile = spark.read.text("/lifeng/student/liutuozhen/streaming/aggregate/table.csv")

In [11]:
textFile.filter(textFile.value.contains("Spark"))

DataFrame[value: string]

In [12]:
textFile.value.contains("Spark")

Column<b'contains(value, Spark)'>

In [13]:
sc = spark.sparkContext

In [14]:
textFile = sc.textFile("/lifeng/student/liutuozhen/streaming/aggregate/table.csv")

In [15]:
def myFunc(s):
    words = s.split(",")
    return len([w for w in words if w != "北京"])

textFile.map(myFunc).collect()

[2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 2]

In [5]:
data = [1, 2, 3, 4, 5]
global counter
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    print(x)
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)

[Stage 0:>                                                          (0 + 2) / 2]

Counter value:  0


In [6]:
def increment_counter(x):
    global counter
    print(counter)
    counter += x
rdd.foreach(increment_counter)

In [7]:
rdd.collect()

[1, 2, 3, 4, 5]

## Create spark_df from RDD

In [10]:
lines = sc.textFile("/lifeng/student/liutuozhen/streaming/aggregate/table.csv")

In [8]:
lines.first()

'北京,女,7'

In [13]:
from pyspark.sql import Row

In [14]:
parts = lines.map(lambda l: l.split(","))
cols = parts.map(lambda p: Row(area=p[0], num=int(p[2])))

In [15]:
spark_df = spark.createDataFrame(cols)
spark_df

DataFrame[area: string, num: bigint]

In [16]:
spark_df.show()

+----+---+
|area|num|
+----+---+
|北京|  7|
|江苏|  8|
|上海| 18|
|北京| 45|
|江苏| 77|
|江苏| 24|
|江苏| 77|
|江苏| 31|
|上海| 71|
|江苏| 57|
|上海| 72|
|浙江| 66|
|北京| 19|
|江苏|  9|
|浙江| 72|
|北京| 16|
|北京| 61|
|北京| 21|
|浙江| 29|
|上海| 34|
+----+---+
only showing top 20 rows



In [18]:
spark_df.createOrReplaceTempView("tempDF") # 建立临时表tempDF
filters = spark.sql("SELECT * FROM tempDF WHERE num>70")
filters.show()

+----+---+
|area|num|
+----+---+
|江苏| 77|
|江苏| 77|
|上海| 71|
|上海| 72|
|浙江| 72|
|上海| 75|
|北京| 76|
|江苏| 72|
|上海| 74|
|江苏| 72|
|浙江| 78|
|上海| 76|
|浙江| 74|
+----+---+



In [20]:
# 无分布式 to Master Node
pdf = filters.toPandas()

In [23]:
df_json = filters.toJSON()
df_json.collect()

['{"area":"江苏","num":77}',
 '{"area":"江苏","num":77}',
 '{"area":"上海","num":71}',
 '{"area":"上海","num":72}',
 '{"area":"浙江","num":72}',
 '{"area":"上海","num":75}',
 '{"area":"北京","num":76}',
 '{"area":"江苏","num":72}',
 '{"area":"上海","num":74}',
 '{"area":"江苏","num":72}',
 '{"area":"浙江","num":78}',
 '{"area":"上海","num":76}',
 '{"area":"浙江","num":74}']

## Create spark_df from file

In [27]:
# local mode: local path / yarn mode: hdfs path
sdf = spark.read.csv("/lifeng/student/liutuozhen/streaming/aggregate/table.csv")
sdf.show()

+----+---+---+
| _c0|_c1|_c2|
+----+---+---+
|北京| 女|  7|
|江苏| 男|  8|
|上海| 男| 18|
|北京| 女| 45|
|江苏| 男| 77|
|江苏| 男| 24|
|江苏| 男| 77|
|江苏| 女| 31|
|上海| 女| 71|
|江苏| 男| 57|
|上海| 女| 72|
|浙江| 男| 66|
|北京| 男| 19|
|江苏| 男|  9|
|浙江| 男| 72|
|北京| 男| 16|
|北京| 女| 61|
|北京| 男| 21|
|浙江| 女| 29|
|上海| 男| 34|
+----+---+---+
only showing top 20 rows



## Create schema for csv header

In [34]:
# Import data types
from pyspark.sql.types import *
# The schema is encoded in a string.

# Create a schema
schemaString = ["area", "gender", "num"]
StringType = [StringType(), StringType(), IntegerType()]
fields = [StructField(field_name, string_type, True) for field_name, string_type in zip(schemaString, StringType)] # True: 是否允许空值
schema = StructType(fields)

In [35]:
schema

StructType(List(StructField(area,StringType,true),StructField(gender,StringType,true),StructField(num,IntegerType,true)))

In [38]:
sdf_with_schema = spark.read.csv("/lifeng/student/liutuozhen/streaming/aggregate/table.csv", schema)
sdf_with_schema.show()

+----+------+---+
|area|gender|num|
+----+------+---+
|北京|    女|  7|
|江苏|    男|  8|
|上海|    男| 18|
|北京|    女| 45|
|江苏|    男| 77|
|江苏|    男| 24|
|江苏|    男| 77|
|江苏|    女| 31|
|上海|    女| 71|
|江苏|    男| 57|
|上海|    女| 72|
|浙江|    男| 66|
|北京|    男| 19|
|江苏|    男|  9|
|浙江|    男| 72|
|北京|    男| 16|
|北京|    女| 61|
|北京|    男| 21|
|浙江|    女| 29|
|上海|    男| 34|
+----+------+---+
only showing top 20 rows



## Convert DF to RDD

In [39]:
sdf_with_schema.rdd

MapPartitionsRDD[95] at javaToPython at NativeMethodAccessorImpl.java:0

## Write DF to local disk

In [41]:
sdf_with_schema.write.mode("overwrite").csv("/lifeng/student/liutuozhen/sdf_save/")

In [42]:
!hdfs dfs -ls /lifeng/student/liutuozhen/sdf_save

Found 2 items
-rw-r-----   2 student hadoop          0 2021-11-10 19:41 /lifeng/student/liutuozhen/sdf_save/_SUCCESS
-rw-r-----   2 student hadoop       1390 2021-11-10 19:41 /lifeng/student/liutuozhen/sdf_save/part-00000-bb23d8ed-5d01-4a90-886e-041bd0535172-c000.csv


## Read file and infer the schema from the header¶

In [7]:
 ## Load a local file becasue we are using spark's local mode
air0 = spark.read.options(header='true', inferSchema='true').csv("/lifeng/student/liutuozhen/airdelay_small.csv")

In [8]:
air0 # the schema is not correct for some variables

DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, DepTime: string, CRSDepTime: int, ArrTime: string, CRSArrTime: int, UniqueCarrier: string, FlightNum: int, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: int, CancellationCode: string, Diverted: int, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [16]:
# We specify the correct schema by hand
from pyspark.sql.types import *
schema_sdf = StructType([
        StructField('Year', IntegerType(), True),
        StructField('Month', IntegerType(), True),
        StructField('DayofMonth', IntegerType(), True),
        StructField('DayOfWeek', IntegerType(), True),
        StructField('DepTime', DoubleType(), True),
        StructField('CRSDepTime', DoubleType(), True),
        StructField('ArrTime', DoubleType(), True),
        StructField('CRSArrTime', DoubleType(), True),
        StructField('UniqueCarrier', StringType(), True),
        StructField('FlightNum', StringType(), True),
        StructField('TailNum', StringType(), True),
        StructField('ActualElapsedTime', DoubleType(), True),
        StructField('CRSElapsedTime',  DoubleType(), True),
        StructField('AirTime',  DoubleType(), True),
        StructField('ArrDelay',  DoubleType(), True),
        StructField('DepDelay',  DoubleType(), True),
        StructField('Origin', StringType(), True),
        StructField('Dest',  StringType(), True),
        StructField('Distance',  DoubleType(), True),
        StructField('TaxiIn',  DoubleType(), True),
        StructField('TaxiOut',  DoubleType(), True),
        StructField('Cancelled',  IntegerType(), True),
        StructField('CancellationCode',  StringType(), True),
        StructField('Diverted',  IntegerType(), True),
        StructField('CarrierDelay', DoubleType(), True),
        StructField('WeatherDelay',  DoubleType(), True),
        StructField('NASDelay',  DoubleType(), True),
        StructField('SecurityDelay',  DoubleType(), True),
        StructField('LateAircraftDelay',  DoubleType(), True)
    ])
air = spark.read.options(header='true').schema(schema_sdf).csv("/lifeng/student/liutuozhen/airdelay_small.csv")

In [17]:
air

DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, DepTime: double, CRSDepTime: double, ArrTime: double, CRSArrTime: double, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: double, CRSElapsedTime: double, AirTime: double, ArrDelay: double, DepDelay: double, Origin: string, Dest: string, Distance: double, TaxiIn: double, TaxiOut: double, Cancelled: int, CancellationCode: string, Diverted: int, CarrierDelay: double, WeatherDelay: double, NASDelay: double, SecurityDelay: double, LateAircraftDelay: double]

## Descriptive Statistics

In [18]:
air.describe().show()

[Stage 10:==============================================>           (4 + 1) / 5]

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------+------------------+-------+------------------+------------------+------------------+-----------------+-----------------+-------+-------+-----------------+------------------+------------------+---------+----------------+--------+------------------+------------------+-----------------+--------------------+-----------------+
|summary|              Year|             Month|        DayofMonth|         DayOfWeek|           DepTime|       CRSDepTime|           ArrTime|        CRSArrTime|UniqueCarrier|         FlightNum|TailNum| ActualElapsedTime|    CRSElapsedTime|           AirTime|         ArrDelay|         DepDelay| Origin|   Dest|         Distance|            TaxiIn|           TaxiOut|Cancelled|CancellationCode|Diverted|      CarrierDelay|      WeatherDelay|         NASDelay|       SecurityDelay|LateAircraftDelay|
+---

In [50]:
air.describe(['ArrDelay']).show()

[Stage 27:==================================================>     (29 + 3) / 32]

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|           5432958|
|   mean|  6.97897995898367|
| stddev|30.191156753519472|
|    min|           -1238.0|
|    max|            1779.0|
+-------+------------------+



In [52]:
air.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: double (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Carrier

In [55]:
air.select(air['UniqueCarrier'], air['ArrDelay']>0).show()

+-------------+--------------+
|UniqueCarrier|(ArrDelay > 0)|
+-------------+--------------+
|           XE|          true|
|           CO|          true|
|           AA|          true|
|           WN|         false|
|           CO|          true|
|           AA|          true|
|           DL|         false|
|           AA|          true|
|           US|          true|
|           AA|          true|
|           AS|         false|
|           UA|          true|
|           TW|         false|
|           NW|          true|
|           NW|          true|
|           AA|          true|
|           DH|          true|
|           WN|         false|
|           AA|         false|
|           CO|          true|
+-------------+--------------+
only showing top 20 rows



In [11]:
## Group and sort
aircount=air.groupBy("UniqueCarrier").count()
aircount.sort("count",ascending=False).show()

21/12/01 19:44:04 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
[Stage 3:================================================>      (175 + 3) / 200]

+-------------+------+
|UniqueCarrier| count|
+-------------+------+
|           DL|765388|
|           WN|703368|
|           AA|684522|
|           US|649056|
|           UA|611957|
|           NW|473820|
|           CO|373858|
|           TW|179081|
|           HP|173509|
|           MQ|164790|
|           AS|129863|
|           OO|120223|
|           XE| 94311|
|           EV| 67148|
|           OH| 60630|
|           FL| 47540|
|           EA| 43723|
|           PI| 41489|
|           DH| 32900|
|           B6| 29111|
+-------------+------+
only showing top 20 rows



In [12]:
air.distinct().count()

1524482

In [ ]:
air.na.drop().show()

In [20]:
air.na.replace("NA", "unknown").show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2006|    7|         6|        4| 2055.0|    2055.0| 2150.0|    2148.0|           XE|     2619

In [23]:
air.groupBy().max('DayOfWeek').show() # mus apply to a numeric column

[Stage 16:==============================================>           (4 + 1) / 5]

+--------------+
|max(DayOfWeek)|
+--------------+
|             7|
+--------------+



In [24]:
air.filter(air.ArrDelay>60).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    5|        17|        4| 1944.0|    1830.0| 2034.0|    1930.0|           AA|     1011

## User-defined functions

In [25]:
air2_pdf = air.select(["DayOfWeek", "ArrDelay","AirTime","Distance"]).toPandas()

In [26]:
air2_pdf.head()

,DayOfWeek,ArrDelay,AirTime,Distance
0,4.0,2.0,25.0,127.0
1,7.0,29.0,248.0,1623.0
2,NaN,NaN,NaN,NaN
3,5.0,-2.0,70.0,451.0
4,7.0,11.0,133.0,1009.0


In [27]:
import pandas as pd

def myfun(pdf):
    out = dict() 
    out["ArrDelay"] = pdf.ArrDelay.mean()
    out["AirTime"]  = pdf.AirTime.mean()
    out["Distance"] = pdf.Distance.mean()
    
    return pd.DataFrame(out, index=[0])

myfun(air2_pdf)

,ArrDelay,AirTime,Distance
0,7.350591,102.688519,729.997977


In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf("DayOfWeek long, ArrDelay long", PandasUDFType.GROUPED_MAP)   # , AirTime long, Distance long
def myfun(pdf):
    out = dict() 
    out["ArrDelay"] = pdf.ArrDelay.mean()
#     out["AirTime"]  = pdf.AirTime.mean()
#     out["Distance"] = pdf.Distance.mean()
    return pd.Dataframe(out, index=[0])

In [29]:
air2 = air.select(["DayOfWeek","ArrDelay","AirTime","Distance"])# .filter(air.Distance<100)

In [30]:
air2.show()

+---------+--------+-------+--------+
|DayOfWeek|ArrDelay|AirTime|Distance|
+---------+--------+-------+--------+
|        4|     2.0|   25.0|   127.0|
|        7|    29.0|  248.0|  1623.0|
|     null|    null|   null|    null|
|        5|    -2.0|   70.0|   451.0|
|        7|    11.0|  133.0|  1009.0|
|        7|    13.0|  177.0|  1562.0|
|        1|   -12.0|  181.0|  1589.0|
|        3|    11.0|  364.0|  2611.0|
|        5|    13.0|   53.0|   304.0|
|     null|    null|   null|    null|
|        5|    -8.0|  293.0|  2537.0|
|     null|    null|   null|    null|
|     null|    null|   null|    null|
|        2|    55.0|  285.0|  1927.0|
|        1|    23.0|  149.0|   991.0|
|        4|    64.0|   35.0|   193.0|
|        4|    29.0|   25.0|    77.0|
|     null|    null|   null|    null|
|        7|    -6.0|   91.0|   678.0|
|        7|    35.0|  127.0|   998.0|
+---------+--------+-------+--------+
only showing top 20 rows



In [ ]:
air3.show()

In [31]:
air3 = air2.na.drop()
air3.select(["DayOfWeek", "ArrDelay"]).groupby("DayOfWeek").apply(myfun).show()

21/12/01 19:55:07 WARN TaskSetManager: Lost task 24.0 in stage 28.0 (TID 478, emr-worker-2.cluster-46968, executor 2): java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.arrow.vector.ipc.message.MessageChannelReader.readNext(MessageChannelReader.java:58)
	at org.apache.arrow.vector.ipc.ArrowStreamReader.readSchema(ArrowStreamReader.java:132)
	at org.apache.arrow.vector.ipc.ArrowReader.initialize(ArrowReader.java:181)
	at org.apache.arrow.vector.ipc.ArrowReader.ensureInitialized(ArrowReader.java:172)
	at org.apache.arrow.vector.ipc.ArrowReader.getVectorSchemaRoot(ArrowReader.java:65)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:122)
	at org.apache.spark.api.python.BasePythonRunner$Reader

Py4JJavaError: An error occurred while calling o241.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 24 in stage 28.0 failed 4 times, most recent failure: Lost task 24.3 in stage 28.0 (TID 486, emr-worker-1.cluster-46968, executor 1): java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.arrow.vector.ipc.message.MessageChannelReader.readNext(MessageChannelReader.java:58)
	at org.apache.arrow.vector.ipc.ArrowStreamReader.readSchema(ArrowStreamReader.java:132)
	at org.apache.arrow.vector.ipc.ArrowReader.initialize(ArrowReader.java:181)
	at org.apache.arrow.vector.ipc.ArrowReader.ensureInitialized(ArrowReader.java:172)
	at org.apache.arrow.vector.ipc.ArrowReader.getVectorSchemaRoot(ArrowReader.java:65)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:162)
	at 

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])


In [27]:
df.show()

+--------------------+
|            features|
+--------------------+
|(4,[0,3],[1.0,-2.0])|
|   [4.0,5.0,0.0,3.0]|
|   [6.0,7.0,0.0,8.0]|
| (4,[0,3],[9.0,1.0])|
+--------------------+



In [28]:
r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

21/11/14 13:43:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/11/14 13:43:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])


21/11/14 13:43:23 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.
